In [5]:
from datetime import datetime
import pandas as pd
import wikipedia as wp
import os # for creating the file path
import requests
from bs4 import BeautifulSoup

timezoneLondon = pytz.timezone('Europe/London')
datetimeLondon = datetime.now(timezoneLondon)
executionTime = datetimeLondon.strftime("_gmt_%d%m%Y_%H%M%S")
directory = './csv/'
fileName = directory+"coronavirus_outbreak_by_country_and_territory"+executionTime+".csv"

if not os.path.isdir(directory):
    os.mkdir(directory) 

# Get the data table source
dataTable = wp.page("2019–20_coronavirus_outbreak_by_country_and_territory").html().encode("UTF-8")
df = pd.read_html(dataTable)[1]
df.to_csv(fileName,header=None,index=False)

print (executionTime)
print (fileName)
print (df)


# This is to get data for UK, can be automated for each country on the table above
# This part is taken from https://johnricco.github.io/2017/04/04/python-html/ - This deals with the row-span and col-span in html
# Added replace('\n','', regex=True) to remove extra carriage return from every cells
class html_tables(object):
    
    def __init__(self, url):
        
        self.url      = url
        self.r        = requests.get(self.url)
        self.url_soup = BeautifulSoup(self.r.text)
        
    def read(self):
        
        self.tables      = []
        self.tables_html = self.url_soup.find_all("table")
        
        # Parse each table
        for n in range(0, len(self.tables_html)):
            
            n_cols = 0
            n_rows = 0
            
            for row in self.tables_html[n].find_all("tr"):
                col_tags = row.find_all(["td", "th"])
                if len(col_tags) > 0:
                    n_rows += 1
                    if len(col_tags) > n_cols:
                        n_cols = len(col_tags)
            
            # Create dataframe
            df = pd.DataFrame(index = range(0, n_rows), columns = range(0, n_cols))
            
            # Create list to store rowspan values 
            skip_index = [0 for i in range(0, n_cols)]
            
            # Start by iterating over each row in this table...
            row_counter = 0
            for row in self.tables_html[n].find_all("tr"):
                
                # Skip row if it's blank
                if len(row.find_all(["td", "th"])) == 0:
                    next
                
                else:
                    
                    # Get all cells containing data in this row
                    columns = row.find_all(["td", "th"])
                    col_dim = []
                    row_dim = []
                    col_dim_counter = -1
                    row_dim_counter = -1
                    col_counter = -1
                    this_skip_index = copy.deepcopy(skip_index)
                    
                    for col in columns:
                        
                        # Determine cell dimensions
                        colspan = col.get("colspan")
                        if colspan is None:
                            col_dim.append(1)
                        else:
                            col_dim.append(int(colspan))
                        col_dim_counter += 1
                            
                        rowspan = col.get("rowspan")
                        if rowspan is None:
                            row_dim.append(1)
                        else:
                            row_dim.append(int(rowspan))
                        row_dim_counter += 1
                            
                        # Adjust column counter
                        if col_counter == -1:
                            col_counter = 0  
                        else:
                            col_counter = col_counter + col_dim[col_dim_counter - 1]
                            
                        while skip_index[col_counter] > 0:
                            col_counter += 1

                        # Get cell contents  
                        cell_data = col.get_text()
                        
                        # Insert data into cell
                        df.iat[row_counter, col_counter] = cell_data

                        # Record column skipping index
                        if row_dim[row_dim_counter] > 1:
                            this_skip_index[col_counter] = row_dim[row_dim_counter]
                
                # Adjust row counter 
                row_counter += 1
                
                # Adjust column skipping index
                skip_index = [i - 1 if i > 0 else i for i in this_skip_index]

            # Append dataframe to list of tables
            self.tables.append(df.replace('\n','', regex=True))
        
        return(self.tables)

datetimeLondon = datetime.now(timezoneLondon)
executionTime = datetimeLondon.strftime("_gmt_%d%m%Y_%H%M%S")
fileName = directory+"coronavirus_outbreak_UK"+executionTime+".csv"

url = "https://en.wikipedia.org/wiki/2020_coronavirus_outbreak_in_the_United_Kingdom"
ukWiki = html_tables(url)
dataTable = ukWiki.read()[4]
dataTable.to_csv(fileName,header=None,index=False)

#print(first_table)

_gmt_06032020_220929
./csv/coronavirus_outbreak_by_country_and_territory_gmt_06032020_220929.csv
                                                    0       1          2  \
0                             Country or territory[a]   Cases     Deaths   
1                             China (mainland)[24][c]   80573       3042   
2                                     South Korea[24]    6593         42   
3                                            Iran[24]    4747        124   
4                                           Italy[25]    4636        197   
5                     International conveyance[24][d]     696          6   
6                                         Germany[24]     670          0   
7                                          France[24]     653          9   
8                                           Japan[24]     420          6   
9                                           Spain[24]     400          5   
10                                  United States[24]     260      